We denote $X \odot Y$ as the (circular) cross-correlation and $X \star  Y$ as the (circular) convolution.

In [6]:
import torch

Circular cross-correlation is defined as
$$
(X \odot Y)[k] = \sum_{n=0}^{N-1} X[n] Y[(k + n) \operatorname{mod} {N}]
$$

In [75]:
siglen = 5
x = ["x" + str(i) for i in range(siglen)]
y = ["y" + str(i) for i in range(siglen)]
for out_idx in range(siglen):
  shflx = [x[i] for i in range(siglen)]
  shfly = [y[i] for i in list(map(lambda x: (out_idx + x) % siglen, range(siglen)))]
  print(*shflx)
  print(*shfly)
  print("=" * 10)

x0 x1 x2 x3 x4
y0 y1 y2 y3 y4
x0 x1 x2 x3 x4
y1 y2 y3 y4 y0
x0 x1 x2 x3 x4
y2 y3 y4 y0 y1
x0 x1 x2 x3 x4
y3 y4 y0 y1 y2
x0 x1 x2 x3 x4
y4 y0 y1 y2 y3


Circular convolution is defined as
$$
(X \star Y)[k] = \sum_{n=0}^{N-1} X[n] Y[(k - n) \operatorname{mod} {N}]
$$

In [76]:
siglen = 5
x = ["x" + str(i) for i in range(siglen)]
y = ["y" + str(i) for i in range(siglen)]
for out_idx in range(siglen):
  shflx = [x[i] for i in range(siglen)]
  shfly = [y[i] for i in list(map(lambda x: (out_idx - x) % siglen, range(siglen)))]
  print(*shflx)
  print(*shfly)
  print("=" * 10)

x0 x1 x2 x3 x4
y0 y4 y3 y2 y1
x0 x1 x2 x3 x4
y1 y0 y4 y3 y2
x0 x1 x2 x3 x4
y2 y1 y0 y4 y3
x0 x1 x2 x3 x4
y3 y2 y1 y0 y4
x0 x1 x2 x3 x4
y4 y3 y2 y1 y0


They can be obtained one from the other. Particularly, let the circular reversal be defined as $\tilde{X}[n] = X[-n \operatorname{mod} N]$. Then, we can obtain cross correlation from convolution by substituting $X \mapsto \tilde{X}$:
$$
(\tilde{X} \star Y)[k] = \sum_{n=0}^{N-1} X[-n] Y[(k - n)] = \sum_{m=0}^{N-1} X[m] Y[k + m] = (X \odot Y)[k],
$$
where indexing is mod $N$, but omitted for brevity. The second equality follows by setting $m = - n$ (summations are preserved since indexing is modular).

In [20]:
def _circ_rev(x):
    N = len(x)
    return [x[(-n) % N] for n in range(N)]
siglen = 5
x = ["x" + str(i) for i in range(siglen)]
y = ["y" + str(i) for i in range(siglen)]
x = _circ_rev(x)
for out_idx in range(siglen):
  shflx = [x[i] for i in range(siglen)]
  shfly = [y[i] for i in list(map(lambda x: (out_idx - x) % siglen, range(siglen)))]
  print(*shflx)
  print(*shfly)
  print("=" * 10)

x0 x4 x3 x2 x1
y0 y4 y3 y2 y1
x0 x4 x3 x2 x1
y1 y0 y4 y3 y2
x0 x4 x3 x2 x1
y2 y1 y0 y4 y3
x0 x4 x3 x2 x1
y3 y2 y1 y0 y4
x0 x4 x3 x2 x1
y4 y3 y2 y1 y0


The other direction is also true:
$$
(\tilde{X} \odot Y)[k] = \sum_{n=0}^{N-1} X[-n] Y[(k - n)] = \sum_{m=0}^{N-1} X[m] Y[k + m] = (X \star Y)[k].
$$

In [2]:
def _circ_rev(x):
    N = len(x)
    return [x[(-n) % N] for n in range(N)]
siglen = 5
x = ["x" + str(i) for i in range(siglen)]
y = ["y" + str(i) for i in range(siglen)]
x = _circ_rev(x)
for out_idx in range(siglen):
  shflx = [x[i] for i in range(siglen)]
  shfly = [y[i] for i in list(map(lambda x: (out_idx + x) % siglen, range(siglen)))]
  print(*shflx)
  print(*shfly)
  print("=" * 10)

x0 x4 x3 x2 x1
y0 y1 y2 y3 y4
x0 x4 x3 x2 x1
y1 y2 y3 y4 y0
x0 x4 x3 x2 x1
y2 y3 y4 y0 y1
x0 x4 x3 x2 x1
y3 y4 y0 y1 y2
x0 x4 x3 x2 x1
y4 y0 y1 y2 y3


In deep learning, we generally use "linear convolutions" (well, actually, we use linear cross-correlations). These can be implemented by padding, performing a circular convolution, and cropping. The following cell performs $W \star_L X$.

In [144]:
x = ["x" + str(i) for i in range(7)]
w = ["w" + str(i) for i in range(3)]
siglen = len(x) + len(w) - 1
x = x + ["0 "] * (siglen - len(x))
w = w + ["0 "] * (siglen - len(w))
for out_idx in range(siglen):
  shflw = [w[i] for i in range(siglen)]
  shflx = [x[i] for i in list(map(lambda x: (out_idx + x) % siglen, range(siglen)))]
  print(*shflw)
  print(*shflx)
  print("=" * 10)

w0 w1 w2 0  0  0  0  0  0 
x0 x1 x2 x3 x4 x5 x6 0  0 
w0 w1 w2 0  0  0  0  0  0 
x1 x2 x3 x4 x5 x6 0  0  x0
w0 w1 w2 0  0  0  0  0  0 
x2 x3 x4 x5 x6 0  0  x0 x1
w0 w1 w2 0  0  0  0  0  0 
x3 x4 x5 x6 0  0  x0 x1 x2
w0 w1 w2 0  0  0  0  0  0 
x4 x5 x6 0  0  x0 x1 x2 x3
w0 w1 w2 0  0  0  0  0  0 
x5 x6 0  0  x0 x1 x2 x3 x4
w0 w1 w2 0  0  0  0  0  0 
x6 0  0  x0 x1 x2 x3 x4 x5
w0 w1 w2 0  0  0  0  0  0 
0  0  x0 x1 x2 x3 x4 x5 x6
w0 w1 w2 0  0  0  0  0  0 
0  x0 x1 x2 x3 x4 x5 x6 0 


Similarly, we can obtain linear cross-correlation by circular reversal of the signal after paddding.

In [9]:
def _circ_rev(x):
    N = len(x)
    return [x[(-n) % N] for n in range(N)]
x = ["x" + str(i) for i in range(7)]
w = ["w" + str(i) for i in range(3)]
siglen = len(x) + len(w) - 1
x = x + ["0 "] * (siglen - len(x))
w = w + ["0 "] * (siglen - len(w))
w = _circ_rev(w)
for out_idx in range(siglen):
  shflw = [w[i] for i in range(siglen)]
  shflx = [x[i] for i in list(map(lambda x: (out_idx - x) % siglen, range(siglen)))]
  print(*shflw)
  print(*shflx)
  print("=" * 10)

w0 0  0  0  0  0  0  w2 w1
x0 0  0  x6 x5 x4 x3 x2 x1
w0 0  0  0  0  0  0  w2 w1
x1 x0 0  0  x6 x5 x4 x3 x2
w0 0  0  0  0  0  0  w2 w1
x2 x1 x0 0  0  x6 x5 x4 x3
w0 0  0  0  0  0  0  w2 w1
x3 x2 x1 x0 0  0  x6 x5 x4
w0 0  0  0  0  0  0  w2 w1
x4 x3 x2 x1 x0 0  0  x6 x5
w0 0  0  0  0  0  0  w2 w1
x5 x4 x3 x2 x1 x0 0  0  x6
w0 0  0  0  0  0  0  w2 w1
x6 x5 x4 x3 x2 x1 x0 0  0 
w0 0  0  0  0  0  0  w2 w1
0  x6 x5 x4 x3 x2 x1 x0 0 
w0 0  0  0  0  0  0  w2 w1
0  0  x6 x5 x4 x3 x2 x1 x0


By the convolution theorem,
$$
X \star Y = \operatorname{Re}(\operatorname{IDFT}(\operatorname{DFT}(X) \cdot \operatorname{DFT}(Y))),
$$
and by noting that $\tilde{X} = \operatorname{DFT}(X)^*$,
$$
X \odot Y = \operatorname{Re}(\operatorname{IDFT}(\operatorname{DFT}(X)^* \cdot \operatorname{DFT}(Y))).
$$
We are interested in linear cross-correlation, matching PyTorch's API.

In [18]:
# First, circular reverse in time space.
def _circ_rev(x):
    N = len(x)
    return torch.tensor([x[(-n) % N] for n in range(N)])

def conv1d(x, w):
  retsize = x.size(0) - w.size(0) + 1
  padlen = x.size(0) + w.size(0) - 1
  padded_x = torch.nn.functional.pad(x, (0, padlen - x.size(0)))
  padded_w = torch.nn.functional.pad(w, (0, padlen - w.size(0)))
  padded_w = _circ_rev(padded_w)
  x_freq = torch.fft.fft(padded_x)
  w_freq = torch.fft.fft(padded_w)
  fullconv = torch.fft.ifft(w_freq * x_freq ).real
  return fullconv[:retsize]
x = torch.tensor([1., 2., 3., 5.0, 2.0])
y = torch.tensor([0., 1., 0.5])
print(conv1d(x, y))
print(torch.nn.functional.conv1d(x.unsqueeze(0).unsqueeze(0), y.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0))

tensor([3.5000, 5.5000, 6.0000])
tensor([3.5000, 5.5000, 6.0000])


In [19]:
# Circular reverse in time space <=> conjugation in frequency space
def conv1d(x, w):
  retsize = x.size(0) - w.size(0) + 1
  padlen = x.size(0) + w.size(0) - 1
  padded_x = torch.nn.functional.pad(x, (0, padlen - x.size(0)))
  padded_w = torch.nn.functional.pad(w, (0, padlen - w.size(0)))
  x_freq = torch.fft.fft(padded_x)
  w_freq = torch.fft.fft(padded_w).conj()
  fullconv = torch.fft.ifft(w_freq * x_freq ).real
  return fullconv[:retsize]

x = torch.tensor([1., 2., 3., 5.0, 2.0])
y = torch.tensor([0., 1., 0.5])
print(conv1d(x, y))
print(torch.nn.functional.conv1d(x.unsqueeze(0).unsqueeze(0), y.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0))

tensor([3.5000, 5.5000, 6.0000])
tensor([3.5000, 5.5000, 6.0000])
